In [11]:
import pandas as pd
import numpy as np
import time,datetime 
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import datasets, metrics, cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
vectorizer = TfidfVectorizer()

In [17]:
# Train data
data_train = pd.read_csv('data_test.csv', header=None)
data_train.columns = ['Sentiment','Text']
data_train = data_train.dropna()

X = np.array(data_train['Text'])
y = np.array(data_train['Sentiment'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

train = vectorizer.fit_transform(X_train)
test = vectorizer.transform(X_test)

In [18]:
classifier_factories = []
classifier_factories.append(lambda: LogisticRegression(C=1.0, penalty='l2'))
classifier_factories.append(lambda: MultinomialNB())
classifier_factories.append(lambda: SVC(C = 1.0, kernel='linear', random_state=241, probability=True))

In [19]:
classifiers = [f().fit(train, y_train) for f in classifier_factories]

In [20]:
pred = np.array([c.predict(test) for c in classifiers]).T
for i in range(len(classifiers)):
    acc = metrics.accuracy_score(y_test,pred[:,i])
    print(i,' ',acc)
pred = pred.astype(int)
prob = np.array([c.predict_proba(test) for c in classifiers]).T

0   0.650243483243
1   0.637639644801
2   0.647951876253


In [21]:
num=0
mas = []
while acc < 0.7:
    for i in range(len(pred)):
        x = (pred[i][0]==pred[i][1]).sum()+(pred[i][0]==pred[i][2])
        if x<=1:
            #print(' VE ',i)
            X_train = np.hstack((X_train,X_test[i]))
            y_train = np.hstack((y_train,y_test[i]))
            mas.append(i)
            num+=1
        else:
            k = stats.distributions.entropy((prob[0][i],prob[1][i]))
            k = k.mean()
            if k >= 0.68:
                #print('KL-d ',i)
                X_train = np.hstack((X_train,X_test[i]))
                y_train = np.hstack((y_train,y_test[i]))
                mas.append(i)
                num+=1
        if num==100:
            X_test = np.delete(X_test,mas)
            y_test = np.delete(y_test,mas)
            mas=[]
            print('Yeap!')
            print('Length___________',train.shape,' ',X_test.shape)
            train = vectorizer.fit_transform(X_train)
            test = vectorizer.transform(X_test)
            
            classifiers = [f().fit(train, y_train) for f in classifier_factories]
            
            
            pred = np.array([c.predict(test) for c in classifiers]).T
            for i in range(len(classifiers)):
                acc = metrics.accuracy_score(y_test,pred[:,i])
                print(i,' ',acc)
            pred = pred.astype(int)
            prob = np.array([c.predict_proba(test) for c in classifiers]).T
            num=0
            break

Yeap!
Length___________ (1495, 6534)   (3391,)
0   0.659687407844
1   0.644057800059
2   0.655558832203
Yeap!
Length___________ (1595, 6880)   (3291,)
0   0.662412640535
1   0.652689152233
2   0.656943178365
Yeap!
Length___________ (1695, 7211)   (3191,)
0   0.668129113131
1   0.661861485428
2   0.661861485428
Yeap!
Length___________ (1795, 7529)   (3091,)
0   0.674862504044
1   0.6696861857
2   0.67518602394
Yeap!
Length___________ (1895, 7874)   (2991,)
0   0.676696756937
1   0.665329321297
2   0.678034102307
Yeap!
Length___________ (1995, 8232)   (2891,)
0   0.682808716707
1   0.681079211346
2   0.679003804912
Yeap!
Length___________ (2095, 8535)   (2791,)
0   0.684342529559
1   0.683984235041
2   0.687208885704
Yeap!
Length___________ (2195, 8870)   (2691,)
0   0.690821256039
1   0.688591601635
2   0.68933481977
Yeap!
Length___________ (2295, 9179)   (2591,)
0   0.69664222308
1   0.695098417599
2   0.692782709379
Yeap!
Length___________ (2395, 9516)   (2491,)
0   0.697310317142
1  